In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from keras.utils.np_utils import to_categorical
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Flatten, Lambda, Activation
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam

train_df = pd.read_json('../input/train.json')
test_df = pd.read_json('../input/test.json')

Using TensorFlow backend.


In [2]:
def std_img(x):
    for i in range(3):
        x[:, :, i] -= np.mean(x[:, :, i].flatten())
        x[:, :, i] /= np.std(x[:, :, i].flatten()) + 1e-7
    return x

def get_image(df):
    '''Create 3-channel 'images'. Return rescale-normalised images.'''
    images = []
    for i, row in df.iterrows():
        # Formulate the bands as 75x75 arrays
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = (band_1 + band_2)/2

        # Rescale
        img = np.dstack([band_1,band_2,band_3])
        img = std_img(img)

        images.append(img)
    return np.array(images)


train_x = get_image(train_df)
test_x = get_image(test_df)

print(train_x.shape,test_x.shape)

(1604, 75, 75, 3) (8424, 75, 75, 3)


In [3]:
y = train_df.is_iceberg.values
print(y[:5])

[0 0 1 0 0]


In [4]:
from keras.callbacks import ModelCheckpoint,LearningRateScheduler
from keras.layers import Input,BatchNormalization
from keras.models import Model

def create_vgg_model():
    img_input = Input(shape=(75,75,3))
    channel_axis = 3
    
    # Block 1
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = Conv2D(64, (3, 3), activation='linear', padding='same', name='block1_conv2')(x)
    x = BatchNormalization(axis=3)(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(128, (3, 3), activation='linear', padding='same', name='block2_conv2')(x)
    x = BatchNormalization(axis=3)(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(256, (3, 3), activation='linear', padding='same', name='block3_conv3')(x)
    x = BatchNormalization(axis=3)(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(512, (3, 3), activation='linear', padding='same', name='block4_conv3')(x)
    x = BatchNormalization(axis=3)(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(512, (3, 3), activation='linear', padding='same', name='block5_conv3')(x)
    x = BatchNormalization(axis=3)(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

    x = Flatten()(x)
    x = Dense(1024,activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(256,activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(1,activation='sigmoid')(x)
    
    return Model(img_input, x, name='inception_v3')
print('model model')


model model


In [5]:
tmp_m = create_vgg_model()
tmp_m.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 75, 75, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 75, 75, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 75, 75, 64)        36928     
_________________________________________________________________
batch_normalization_1 (Batch (None, 75, 75, 64)        256       
_________________________________________________________________
activation_1 (Activation)    (None, 75, 75, 64)        0         
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 37, 37, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 37, 37, 128)       73856     
__________

In [7]:
from sklearn.model_selection import KFold
from keras.preprocessing.image import ImageDataGenerator


def lr_f(epoch):
    if epoch<20:
        return 0.0005
    elif epoch<50:
        return 0.00005
    else:
        return 0.00001

def kfold_train(fold_cnt=3,rnd=9):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.05,
            height_shift_range=0.05,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 32
        steps_train = len(curr_y)//bat_size
        
        
        model = create_vgg_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.001), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=80, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=4)

Epoch 1/80
Epoch 00001: val_loss improved from inf to 2.76343, saving model to best_m.h5
 - 14s - loss: 1.1805 - acc: 0.5446 - val_loss: 2.7634 - val_acc: 0.5337
Epoch 2/80
Epoch 00002: val_loss improved from 2.76343 to 0.54881, saving model to best_m.h5
 - 13s - loss: 0.5015 - acc: 0.7905 - val_loss: 0.5488 - val_acc: 0.5736
Epoch 3/80
Epoch 00003: val_loss did not improve
 - 12s - loss: 0.4238 - acc: 0.8209 - val_loss: 0.6904 - val_acc: 0.6459
Epoch 4/80
Epoch 00004: val_loss did not improve
 - 12s - loss: 0.3979 - acc: 0.8367 - val_loss: 4.7557 - val_acc: 0.5337
Epoch 5/80
Epoch 00005: val_loss did not improve
 - 12s - loss: 0.4016 - acc: 0.8426 - val_loss: 2.2915 - val_acc: 0.5337
Epoch 6/80
Epoch 00006: val_loss did not improve
 - 12s - loss: 0.3921 - acc: 0.8364 - val_loss: 5.2661 - val_acc: 0.5337
Epoch 7/80
Epoch 00007: val_loss improved from 0.54881 to 0.47141, saving model to best_m.h5
 - 12s - loss: 0.3686 - acc: 0.8446 - val_loss: 0.4714 - val_acc: 0.7656
Epoch 8/80
Epoch 0

Epoch 64/80
Epoch 00064: val_loss did not improve
 - 12s - loss: 0.1793 - acc: 0.9293 - val_loss: 0.2960 - val_acc: 0.8628
Epoch 65/80
Epoch 00065: val_loss did not improve
 - 12s - loss: 0.1597 - acc: 0.9335 - val_loss: 0.2859 - val_acc: 0.8703
Epoch 66/80
Epoch 00066: val_loss did not improve
 - 12s - loss: 0.1838 - acc: 0.9197 - val_loss: 0.2822 - val_acc: 0.8653
Epoch 67/80
Epoch 00067: val_loss did not improve
 - 12s - loss: 0.1956 - acc: 0.9237 - val_loss: 0.2717 - val_acc: 0.8803
Epoch 68/80
Epoch 00068: val_loss did not improve
 - 12s - loss: 0.1676 - acc: 0.9344 - val_loss: 0.2817 - val_acc: 0.8653
Epoch 69/80
Epoch 00069: val_loss did not improve
 - 12s - loss: 0.1477 - acc: 0.9324 - val_loss: 0.2962 - val_acc: 0.8628
Epoch 70/80
Epoch 00070: val_loss did not improve
 - 12s - loss: 0.1757 - acc: 0.9313 - val_loss: 0.2800 - val_acc: 0.8703
Epoch 71/80
Epoch 00071: val_loss did not improve
 - 12s - loss: 0.1743 - acc: 0.9327 - val_loss: 0.2698 - val_acc: 0.8728
Epoch 72/80
Epoc

 - 13s - loss: 0.2226 - acc: 0.9096 - val_loss: 0.1788 - val_acc: 0.9277
Epoch 47/80
Epoch 00047: val_loss did not improve
 - 12s - loss: 0.1906 - acc: 0.9127 - val_loss: 0.2037 - val_acc: 0.9252
Epoch 48/80
Epoch 00048: val_loss did not improve
 - 12s - loss: 0.2106 - acc: 0.9197 - val_loss: 0.1901 - val_acc: 0.9252
Epoch 49/80
Epoch 00049: val_loss improved from 0.17880 to 0.17471, saving model to best_m.h5
 - 13s - loss: 0.2104 - acc: 0.9189 - val_loss: 0.1747 - val_acc: 0.9352
Epoch 50/80
Epoch 00050: val_loss did not improve
 - 12s - loss: 0.2259 - acc: 0.9119 - val_loss: 0.1808 - val_acc: 0.9352
Epoch 51/80
Epoch 00051: val_loss did not improve
 - 13s - loss: 0.2147 - acc: 0.9186 - val_loss: 0.2077 - val_acc: 0.9227
Epoch 52/80
Epoch 00052: val_loss did not improve
 - 13s - loss: 0.2112 - acc: 0.9172 - val_loss: 0.1922 - val_acc: 0.9401
Epoch 53/80
Epoch 00053: val_loss did not improve
 - 12s - loss: 0.1849 - acc: 0.9259 - val_loss: 0.2247 - val_acc: 0.9102
Epoch 54/80
Epoch 0005

Epoch 30/80
Epoch 00030: val_loss improved from 0.24558 to 0.22410, saving model to best_m.h5
 - 12s - loss: 0.2236 - acc: 0.9062 - val_loss: 0.2241 - val_acc: 0.9077
Epoch 31/80
Epoch 00031: val_loss did not improve
 - 12s - loss: 0.2051 - acc: 0.9130 - val_loss: 0.2309 - val_acc: 0.9177
Epoch 32/80
Epoch 00032: val_loss did not improve
 - 12s - loss: 0.2026 - acc: 0.9146 - val_loss: 0.2306 - val_acc: 0.9077
Epoch 33/80
Epoch 00033: val_loss did not improve
 - 12s - loss: 0.2130 - acc: 0.9046 - val_loss: 0.2351 - val_acc: 0.9027
Epoch 34/80
Epoch 00034: val_loss did not improve
 - 12s - loss: 0.1974 - acc: 0.9217 - val_loss: 0.2457 - val_acc: 0.9027
Epoch 35/80
Epoch 00035: val_loss improved from 0.22410 to 0.22183, saving model to best_m.h5
 - 13s - loss: 0.2187 - acc: 0.9096 - val_loss: 0.2218 - val_acc: 0.9102
Epoch 36/80
Epoch 00036: val_loss did not improve
 - 12s - loss: 0.1996 - acc: 0.9209 - val_loss: 0.2363 - val_acc: 0.9052
Epoch 37/80
Epoch 00037: val_loss did not improve
 

Epoch 13/80
Epoch 00013: val_loss did not improve
 - 12s - loss: 0.3234 - acc: 0.8665 - val_loss: 2.4903 - val_acc: 0.5387
Epoch 14/80
Epoch 00014: val_loss did not improve
 - 12s - loss: 0.3051 - acc: 0.8645 - val_loss: 0.5876 - val_acc: 0.7606
Epoch 15/80
Epoch 00015: val_loss improved from 0.26434 to 0.23988, saving model to best_m.h5
 - 12s - loss: 0.3218 - acc: 0.8648 - val_loss: 0.2399 - val_acc: 0.8803
Epoch 16/80
Epoch 00016: val_loss did not improve
 - 12s - loss: 0.2919 - acc: 0.8784 - val_loss: 1.0471 - val_acc: 0.6185
Epoch 17/80
Epoch 00017: val_loss did not improve
 - 12s - loss: 0.3562 - acc: 0.8518 - val_loss: 3.2135 - val_acc: 0.5387
Epoch 18/80
Epoch 00018: val_loss did not improve
 - 12s - loss: 0.3215 - acc: 0.8657 - val_loss: 0.8671 - val_acc: 0.6534
Epoch 19/80
Epoch 00019: val_loss did not improve
 - 12s - loss: 0.3044 - acc: 0.8842 - val_loss: 2.4905 - val_acc: 0.5387
Epoch 20/80
Epoch 00020: val_loss did not improve
 - 12s - loss: 0.2513 - acc: 0.8809 - val_los

Epoch 78/80
Epoch 00078: val_loss did not improve
 - 12s - loss: 0.1994 - acc: 0.9200 - val_loss: 0.1932 - val_acc: 0.8978
Epoch 79/80
Epoch 00079: val_loss did not improve
 - 12s - loss: 0.2038 - acc: 0.9223 - val_loss: 0.1770 - val_acc: 0.9302
Epoch 80/80
Epoch 00080: val_loss did not improve
 - 12s - loss: 0.2077 - acc: 0.9172 - val_loss: 0.1852 - val_acc: 0.9352


In [8]:
import pickle
with open('../features/vgg_aug1_new_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
from sklearn.metrics import log_loss
print(log_loss(y,train_pred))
    
submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/vgg_aug1_new_sub.csv', index=False)

# pre 2370
# new 2088
# new 2033

0.203342999676
         id  is_iceberg
0  5941774d    0.047410
1  4023181e    0.856189
2  b20200e4    0.075777
3  e7f018bb    0.995969
4  4371c8c3    0.096559
